# Lab #2

In [ ]:
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, ObjectDetector

## Camera

Let's start with displaying the output of our camera. Initialize the camera with 300x300 pixels and link it to the widget for live stream.

In [ ]:
camera = Camera.instance(width=300, height=300)


In [ ]:
# display live camera feed
image_widget = ipywidgets.Image()
cam_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
display(image_widget)

Now, let's capture an image with the camera and save it.

In [ ]:
with open('test.jpg', 'wb') as f:
    f.write(image_widget.value)

cam_link.unlink()

# display image
image_widget = ipywidgets.Image()
image_widget.value = open('test.jpg', 'rb').read()
display(image_widget)

## Exploration

In this example we'll show how you can classify objects with JetBot! We'll use a pre-trained neural network that was trained on the COCO dataset to detect 90 different common objects. These include:

- Person (index 0)
- Cup (index 47)

First, we'll want to import the ObjectDetector class which takes our pre-trained SSD engine.

In [ ]:
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Next, let's initialize our camera. Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

In [ ]:
camera = Camera.instance(width=300, height=300)

Now, let's execute our network using some camera input.

In [ ]:
detections = model(camera.value)
print(detections)

If there are any COCO objects in the camera's field of view, they should now be stored in the detections variable.

We'll use the code below to print out the detected objects.

In [ ]:
detections_widget = widgets.Textarea()
detections_widget.value = str(detections)
display(detections_widget)

## Conclusion
- 